In [34]:
import cv2 as cv
import os

def naloziSlike(mapa):
    images = []
    for filename in os.listdir(mapa):
        img = cv.imread(os.path.join(mapa, filename))
        if img is not None:
            images.append(img)
    return images

#Nalozimo slike
slike = naloziSlike("slike")
slike_z_legendo = naloziSlike("slike_z_legendo")

print (slike[0].shape)
print (slike[1].shape)
#Slike niso iste velikosti

(749, 1117, 3)
(745, 1115, 3)


## ZELIM OBREZAT SLIKO, DA OSTANE SAMO VLOZEK, OD ROBA DO ROBA

In [35]:
def remove_border(img):
    grayscale = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    ret, thresholded = cv.threshold(grayscale, 127, 255, cv.THRESH_BINARY_INV)
    cv.imwrite("thresholded.png", thresholded)
    bbox = cv.boundingRect(thresholded)
    x, y, w, h = bbox
    # print(bbox)
    foreground = img[y:y+h, x:x+w]
    cv.imwrite("foreground.png", foreground)
    return foreground

def odstrani_legendo(img):
    # Odstrani 100px navpicni pas na desni strani, kjer je legenda.
    return img[:,:len(img[1]) - 100]

slike_no_border = []
for slika in slike:
    slike_no_border.append(remove_border(slika))

for slika in slike_z_legendo:
    slika = odstrani_legendo(slika)
    slike_no_border.append(remove_border(slika))

In [36]:
import numpy as np

# Vse slike nardimo velikosti kot je prva slika
visina = len(slike_no_border[0])
sirina = len(slike_no_border[0][0])
for i in range(len(slike_no_border)):
    slike_no_border[i] = cv.resize(slike_no_border[i], (sirina, visina))
cv.imwrite("resized.png", slike_no_border[3])

# Odstranimo modro barvo iz vseh razen prve slike
# lowerValues = np.array([33, 27, 153])
# upperValues = np.array([33, 27, 153])
# cv.imwrite("noborder1.png", slike_no_border[1])
# # 53, 40, 255
# mask = cv.inRange(slike_no_border[1], lowerValues, upperValues)
# slike_no_blue = cv.bitwise_and(slike_no_border[1], slike_no_border[1], mask=mask)
# cv.imwrite("mask.png", mask)
# cv.imwrite("noblue.png", slike_no_blue)

def blend(list_images): # Blend images equally.
    equal_fraction = 1.0 / (len(list_images))
    output = np.zeros_like(list_images[0])
    for img in list_images:
        output = output + img * equal_fraction
    output = output.astype(np.uint8)
    return output


result = blend(slike_no_border)
cv.imwrite("zdruzeno.png", result)

# result = cv.addWeighted(slike_no_border[0], 0.5, slike_no_border[1], 0.5, 0)
# for i in range(len(slike_no_border)):
#     result = cv.addWeighted(result, 0.5, slike_no_border[1], 0.5, 0)
#     cv.imwrite("zdruzeno.png", result)
# cv.namedWindow('result',cv.WINDOW_NORMAL)#show the image on the screen
# cv.imshow('result',result)
# if cv.waitKey()==0:
#     cv.destroyAllWindows()

True